# 导入必要的包

In [1]:
import numpy as np
from sympy import *
import math
import matplotlib.pyplot as plt
import mpl_toolkits.axisartist as axisartist
from scipy.optimize import minimize
from numpy import *
import sys
import time

# 线搜索方法

## 精确线搜索 

In [2]:
def step_accurate(g, x, alpha, p):
    cond = np.dot(g(x + alpha * p),p) == 0
    return cond

## 强Wolfe准则

In [3]:
def step_wolfe(f, g, x, alpha, p, rho,sigma):
    result = f(x + alpha * p)
    first_cond = abs(np.dot(g(x + alpha * p),p)) <= -sigma*np.dot(g(x),p)
    second_cond = result <= f(x) + rho * alpha * np.dot(g(x), p)
    return first_cond and second_cond

## Armijo准则

In [4]:
def armijo(gk,dk,x0,p,sigma):
    return fun(x0+p*dk) < fun(x0) + sigma*p*np.dot(gk,dk)
#因为有的优化算法使用精确线搜索一直没有收敛，使用Armijo准则加以比较

## Goldstein 条件

In [5]:
def step_goldstein(f,g,x,alpha,p,rho):
    result = f(x + alpha * p)
    first_cond = f(x) + (1-rho) * alpha * np.matmul(g(x).T,p) <= result
    second_cond = result <= f(x) + rho * alpha * np.matmul(g(x).T, p)
    return first_cond and second_cond
#为简便比较过程，仅以强Wolfe准则和Armijo准则作为非精确线搜索方法的代表

# 最速下降方法

In [6]:
def gradient(fun,gfun,x0):
    #最速下降法求解无约束问题
    # x0是初始点，fun和gfun分别是目标函数和梯度
    start = time.process_time()
    maxk = 5000
    rho = 0.5
    sigma = 0.4
    k = 0
    epsilon = 1e-5
 
    while k<maxk:
        gk = gfun(x0)
        dk = -gk
        if np.linalg.norm(dk) < epsilon:
            break
        m = 0
        mk = 0
        while m< 20:
            if armijo(gk,dk,x0,rho**m,sigma):
            #if step_accurate(gfun, x0, mk, dk):
                mk = m
                break
            m += 1
        x0 += rho**mk*dk
        k += 1
        end = time.process_time()
    print("==================================================================")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    #return x0,fun(x0),k

#gradient(fun,gfun,[0,0])

# 阻尼Newton方法

## 阻尼Newton方法——精确线搜索

In [7]:
def dampnm_acc(fun,gfun,hess,x0):
    #import time
    start = time.process_time()
    # 用阻尼牛顿法求解无约束问题
    # x0是初始点，fun，gfun和hess分别是目标函数值，梯度，海森矩阵的函数
    maxk = 500
    rho = 0.55
    sigma = 0.4
    k = 0
    epsilon = 1e-5
    while k < maxk:
        gk = gfun(x0)
        Gk = hess(x0)
        dk = -1.0*np.linalg.solve(Gk,gk)
        if np.linalg.norm(dk) < epsilon: # ord：范数类型, 默认是二范数
            break
        m = 0
        mk = 0
        while m < 20:
            #if fun(x0+rho**m*dk) < fun(x0) + sigma*rho**m*np.dot(gk,dk):
            if step_accurate(gfun, x0, rho**m, dk):
                mk = m
                break
            m += 1
        x0 += rho**mk*dk
        k += 1
        end = time.process_time()
    print("阻尼Newton方法——精确线搜索:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

## 阻尼Newton方法——强Wolf准则

In [8]:
def dampnm_wol(fun,gfun,hess,x0):
    # 用阻尼牛顿法求解无约束问题
    #x0是初始点，fun，gfun和hess分别是目标函数值，梯度，海森矩阵的函数
    start = time.process_time()
    maxk = 500
    rho = 0.55
    sigma = 0.4
    k = 0
    epsilon = 1e-5
    while k < maxk:
        gk = gfun(x0)
        Gk = hess(x0)
        dk = -1.0*np.linalg.solve(Gk,gk)
        if np.linalg.norm(dk) < epsilon: # ord：范数类型, 默认是二范数
            break
        m = 0
        mk = 0
        while m < 20:
            #if fun(x0+rho**m*dk) < fun(x0) + sigma*rho**m*np.dot(gk,dk):
            if step_wolfe(fun, gfun, x0, rho**m, dk, 0.3,sigma):
                mk = m
                break
            m += 1
        x0 += rho**mk*dk
        k += 1
        end = time.process_time()
    print("阻尼Newton方法——强Wolf准则:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

## 阻尼Newton方法——Armijo准则

In [9]:
def dampnm_arm(fun,gfun,hess,x0):
    # 用阻尼牛顿法求解无约束问题
    # x0是初始点，fun，gfun和hess分别是目标函数值，梯度，海森矩阵的函数
    start = time.process_time()
    maxk = 500
    rho = 0.55
    sigma = 0.4
    k = 0
    epsilon = 1e-5
    while k < maxk:
        gk = gfun(x0)
        Gk = hess(x0)
        dk = -1.0*np.linalg.solve(Gk,gk)
        if np.linalg.norm(dk) < epsilon: # ord：范数类型, 默认是二范数
            break
        m = 0
        mk = 0
        while m < 20:
            if armijo(gk,dk,x0,rho**m,sigma):
            #if step_wolfe(fun, gfun, x0, rho**m, dk, 0.3,sigma):
                mk = m
                break
            m += 1
        x0 += rho**mk*dk
        k += 1
        end = time.process_time()
    print("阻尼Newton方法——Armijo准则:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

# 修正Newton方法

## 修正Newton方法——精确线搜索

In [10]:
def revisenm_acc(fun,gfun,hess,x0):
    # 用修正牛顿法求解无约束问题
    #x0是初始点，fun，gfun和hess分别是目标函数值，梯度，海森矩阵的函数
    start = time.process_time()
    maxk = 1e5
    n = np.shape(x0)[0]
    rho = 0.55
    sigma = 0.4
    tau = 0.0
    k = 0
    epsilon = 1e-5
 
    while k < maxk:
        gk = gfun(x0)      
        if  np.linalg.norm(gk) < epsilon:
            break
        muk = np.power(np.linalg.norm(gk),1+tau)
        Gk = hess(x0)
        Ak = Gk + muk*np.eye(n)
        dk = -1.0*np.linalg.solve(Ak,gk)
        m = 0
        mk = 0
        while m < 20:
            #if fun(x0+rho**m*dk) < fun(x0) + sigma*rho**m*np.dot(gk,dk):
            if step_accurate(gfun, x0, rho**m, dk):
                mk = m
                break
            m += 1
        x0 += rho**mk*dk
        k += 1
        end = time.process_time()
    print("修正Newton方法——精确线搜索:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

## 修正Newton方法——强Wolf准则 

In [11]:
def revisenm_wol(fun,gfun,hess,x0):
    # 用修正牛顿法求解无约束问题
    #x0是初始点，fun，gfun和hess分别是目标函数值，梯度，海森矩阵的函数
    start = time.process_time()
    maxk = 1e5
    n = np.shape(x0)[0]
    rho = 0.55
    sigma = 0.4
    tau = 0.0
    k = 0
    epsilon = 1e-5
 
    while k < maxk:
        gk = gfun(x0)      
        if  np.linalg.norm(gk) < epsilon:
            break
        muk = np.power(np.linalg.norm(gk),1+tau)
        Gk = hess(x0)
        Ak = Gk + muk*np.eye(n)
        dk = -1.0*np.linalg.solve(Ak,gk)
        m = 0
        mk = 0
        while m < 20:
            #if fun(x0+rho**m*dk) < fun(x0) + sigma*rho**m*np.dot(gk,dk):
            if step_wolfe(fun, gfun, x0, rho**m, dk, 0.3,sigma):
                mk = m
                break
            m += 1
        x0 += rho**mk*dk
        k += 1
        end = time.process_time()
    print("修正Newton方法——强Wolf准则:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

## 修正Newton方法——Armijo准则

In [12]:
def revisenm_arm(fun,gfun,hess,x0):
    # 用修正牛顿法求解无约束问题
    #x0是初始点，fun，gfun和hess分别是目标函数值，梯度，海森矩阵的函数
    start = time.process_time()
    maxk = 1e5
    n = np.shape(x0)[0]
    rho = 0.55
    sigma = 0.4
    tau = 0.0
    k = 0
    epsilon = 1e-5
 
    while k < maxk:
        gk = gfun(x0)      
        if  np.linalg.norm(gk) < epsilon:
            break
        muk = np.power(np.linalg.norm(gk),1+tau)
        Gk = hess(x0)
        Ak = Gk + muk*np.eye(n)
        dk = -1.0*np.linalg.solve(Ak,gk)
        m = 0
        mk = 0
        while m < 20:
            if armijo(gk,dk,x0,rho**m,sigma):
            #if step_wolfe(fun, gfun, x0, rho**m, dk, 0.3,sigma):
                mk = m
                break
            m += 1
        x0 += rho**mk*dk
        k += 1
        end = time.process_time()
    print("修正Newton方法——Armijo准则:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

# SR1

## SR1——精确线搜索

In [13]:
def sr1_acc(fun,gfun,hess,x0):
    #功能：用SR1算法求解无约束问题：min fun(x)
    #输入：x0是初始点，fun,gfun分别是目标函数和梯度
    #输出：x,val分别是近似最优点和最优解,k是迭代次数  
    start = time.process_time()
    maxk = 1e5
    rho = 0.55
    sigma = 0.4
    epsilon = 1e-5
    k = 0
    n = np.shape(x0)[0]
    #海森矩阵可以初始化为单位矩阵
    Bk = np.eye(n) #np.linalg.inv(hess(x0))
 
    while k < maxk:
        gk = gfun(x0)
        if np.linalg.norm(gk) < epsilon:
            break
        dk = -1.0*np.linalg.solve(Bk,gk)
        m = 0
        mk = 0
        while m < 20: 
            if step_accurate(gfun, x0, rho**m, dk):
                mk = m
                break
            m += 1
 
        x = x0 + rho**mk*dk
        sk = x - x0
        yk = gfun(x) - gk   
 
        if np.dot(sk,yk) > 0:    
            Bs = np.dot(Bk,sk)
            ys = yk-Bs
 
            Bk = Bk +np.dot((ys).reshape((len(ys),1)),(ys).reshape((1,len(ys))))/np.dot(ys,sk)
 
        k += 1
        x0 = x
        end = time.process_time()
    print("SR1——精确线搜索:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

## SR1——强Wolf准则

In [14]:
def sr1_wol(fun,gfun,hess,x0):
    #功能：用SR1算法求解无约束问题：min fun(x)
    #输入：x0是初始点，fun,gfun分别是目标函数和梯度
    #输出：x,val分别是近似最优点和最优解,k是迭代次数  
    start = time.process_time()
    maxk = 1e5
    rho = 0.55
    sigma = 0.4
    epsilon = 1e-5
    k = 0
    n = np.shape(x0)[0]
    #海森矩阵可以初始化为单位矩阵
    Bk = np.eye(n) #np.linalg.inv(hess(x0))
 
    while k < maxk:
        gk = gfun(x0)
        if np.linalg.norm(gk) < epsilon:
            break
        dk = -1.0*np.linalg.solve(Bk,gk)
        m = 0
        mk = 0
        while m < 20: 
            if step_wolfe(fun, gfun, x0, rho**m, dk, 0.3,sigma):
                mk = m
                break
            m += 1
 
        x = x0 + rho**mk*dk
        sk = x - x0
        yk = gfun(x) - gk   
 
        if np.dot(sk,yk) > 0:    
            Bs = np.dot(Bk,sk)
            ys = yk-Bs
 
            Bk = Bk +np.dot((ys).reshape((len(ys),1)),(ys).reshape((1,len(ys))))/np.dot(ys,sk)
 
        k += 1
        x0 = x
        end = time.process_time()
    print("SR1——强Wolf准则:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

## SR1——Armijo准则

In [15]:
def sr1_arm(fun,gfun,hess,x0):
    #功能：用SR1算法求解无约束问题：min fun(x)
    #输入：x0是初始点，fun,gfun分别是目标函数和梯度
    #输出：x,val分别是近似最优点和最优解,k是迭代次数  
    start = time.process_time()
    maxk = 1e5
    rho = 0.55
    sigma = 0.4
    epsilon = 1e-5
    k = 0
    n = np.shape(x0)[0]
    #海森矩阵可以初始化为单位矩阵
    Bk = np.eye(n) #np.linalg.inv(hess(x0))
 
    while k < maxk:
        gk = gfun(x0)
        if np.linalg.norm(gk) < epsilon:
            break
        dk = -1.0*np.linalg.solve(Bk,gk)
        m = 0
        mk = 0
        while m < 20: 
            if armijo(gk,dk,x0,rho**m,sigma):
                mk = m
                break
            m += 1
 
        x = x0 + rho**mk*dk
        sk = x - x0
        yk = gfun(x) - gk   
 
        if np.dot(sk,yk) > 0:    
            Bs = np.dot(Bk,sk)
            ys = yk-Bs
 
            Bk = Bk +np.dot((ys).reshape((len(ys),1)),(ys).reshape((1,len(ys))))/np.dot(ys,sk)
 
        k += 1
        x0 = x
        end = time.process_time()
    print("SR1——Armijo准则:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

# BFGS

## BFGS——精确线搜索

In [16]:
def bfgs_acc(fun,gfun,hess,x0):
    #功能：用BFGS族算法求解无约束问题：min fun(x)
    #输入：x0是初始点，fun,gfun分别是目标函数和梯度
    #输出：x,val分别是近似最优点和最优解,k是迭代次数  
    start = time.process_time()  
    maxk = 1e5
    rho = 0.55
    sigma = 0.4
    epsilon = 1e-5
    k = 0
    n = np.shape(x0)[0]
    #海森矩阵可以初始化为单位矩阵
    Bk = np.eye(n) #np.linalg.inv(hess(x0))
 
    while k < maxk:
        gk = gfun(x0)
        if np.linalg.norm(gk) < epsilon:
            break
        dk = -1.0*np.linalg.solve(Bk,gk)
        m = 0
        mk = 0
        while m < 20: 
            if step_accurate(gfun, x0, rho**m, dk):
                mk = m
                break
            m += 1
 
        #BFGS校正
        x = x0 + rho**mk*dk
        sk = x - x0
        yk = gfun(x) - gk   
 
        if np.dot(sk,yk) > 0:    
            Bs = np.dot(Bk,sk)
            ys = np.dot(yk,sk)
            sBs = np.dot(np.dot(sk,Bk),sk) 
 
            #Bk = Bk - 1.0*Bs.reshape((n,1))*Bs/sBs + 1.0*yk.reshape((n,1))*yk/ys
            Bk = Bk - np.dot(Bs.reshape((n,1)),Bs.reshape((1,n)))/sBs + np.dot(yk.reshape((n,1)),yk.reshape((1,n)))/ys
 
 
        k += 1
        x0 = x
        end = time.process_time()
    print("BFGS——精确线搜索:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

## BFGS——强Wolf准则

In [17]:
def bfgs_wol(fun,gfun,hess,x0):
    #功能：用BFGS族算法求解无约束问题：min fun(x)
    #输入：x0是初始点，fun,gfun分别是目标函数和梯度
    #输出：x,val分别是近似最优点和最优解,k是迭代次数  
    start = time.process_time()  
    maxk = 1e5
    rho = 0.55
    sigma = 0.4
    epsilon = 1e-5
    k = 0
    n = np.shape(x0)[0]
    #海森矩阵可以初始化为单位矩阵
    Bk = eye(n)#np.linalg.inv(hess(x0)) #或者单位矩阵np.eye(n)
 
    while k < maxk:
        gk = gfun(x0)
        if np.linalg.norm(gk) < epsilon:
            break
        dk = -1.0*np.linalg.solve(Bk,gk)
        m = 0
        mk = 0
        while m < 20: 
            if step_wolfe(fun, gfun, x0, rho**m, dk, 0.3,sigma):
                mk = m
                break
            m += 1
 
        #BFGS校正
        x = x0 + rho**mk*dk
        sk = x - x0
        yk = gfun(x) - gk   
 
        if np.dot(sk,yk) > 0:    
            Bs = np.dot(Bk,sk)
            ys = np.dot(yk,sk)
            sBs = np.dot(np.dot(sk,Bk),sk) 
 
            Bk = Bk - np.dot(Bs.reshape((n,1)),Bs.reshape((1,n)))/sBs + np.dot(yk.reshape((n,1)),yk.reshape((1,n)))/ys
 
        k += 1
        x0 = x
 
        end = time.process_time()
    print("BFGS——强Wolf准则:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'.format(count_G, count_g, count_f))

## BFGS——Armijo准则

In [18]:
def bfgs_arm(fun,gfun,hess,x0):
    #功能：用BFGS族算法求解无约束问题：min fun(x)
    #输入：x0是初始点，fun,gfun分别是目标函数和梯度
    #输出：x,val分别是近似最优点和最优解,k是迭代次数  
    start = time.process_time()  
    maxk = 1e5
    rho = 0.55
    sigma = 0.4
    epsilon = 1e-5
    k = 0
    n = np.shape(x0)[0]
    #海森矩阵可以初始化为单位矩阵
    Bk = eye(n)#np.linalg.inv(hess(x0)) #或者单位矩阵np.eye(n)
 
    while k < maxk:
        gk = gfun(x0)
        if np.linalg.norm(gk) < epsilon:
            break
        dk = -1.0*np.linalg.solve(Bk,gk)
        m = 0
        mk = 0
        while m < 20: 
            if armijo(gk,dk,x0,rho**m,sigma):
                mk = m
                break
            m += 1
 
        #BFGS校正
        x = x0 + rho**mk*dk
        sk = x - x0
        yk = gfun(x) - gk   
 
        if np.dot(sk,yk) > 0:    
            Bs = np.dot(Bk,sk)
            ys = np.dot(yk,sk)
            sBs = np.dot(np.dot(sk,Bk),sk) 
 
            Bk = Bk - np.dot(Bs.reshape((n,1)),Bs.reshape((1,n)))/sBs + np.dot(yk.reshape((n,1)),yk.reshape((1,n)))/ys
 
        k += 1
        x0 = x
        end = time.process_time()
    print("BFGS——Armijo准则:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'.format(count_G, count_g, count_f))

# DFP

## DFP——精确线搜索

In [19]:
def dfp_acc(fun,gfun,hess,x0):
    #功能：用DFP族算法求解无约束问题：min fun(x)
    #输入：x0是初始点，fun,gfun分别是目标函数和梯度
    #输出：x,val分别是近似最优点和最优解,k是迭代次数
    start = time.process_time()
    maxk = 1e3
    rho = 0.55
    sigma = 0.8
    epsilon = 1e-5
    k = 0
    n = np.shape(x0)[0]
    #海森矩阵可以初始化为单位矩阵
    Hk = np.eye(n) #np.linalg.inv(hess(x0))
 
    while k < maxk :
        gk = gfun(x0)
        if np.linalg.norm(gk) < epsilon:
            break
        dk = -1.0*np.dot(Hk,gk)
 
        m=0;
        mk=0
        while m < 20:
            if step_accurate(gfun, x0, rho**m, dk):
                mk = m
                break
            m += 1
        #print mk
        #DFP校正
        x = x0 + rho**mk*dk
        sk = x - x0
        yk = gfun(x) - gk   
 
        if np.dot(sk,yk) > 0:
            Hy = np.dot(Hk,yk)
            #print (Hy)
            sy = np.dot(sk,yk) #向量的点积
            yHy = np.dot(np.dot(yk,Hk),yk) # yHy是标量
            #表达式Hy.reshape((n,1))*Hy 中Hy是向量，生成矩阵
            Hk = Hk - np.dot(Hy.reshape((len(Hy),1)),Hy.reshape((1,len(Hy))))/yHy + np.dot(sk.reshape((len(sk),1)),sk.reshape((1,len(sk))))/sy
 
        k += 1
        x0 = x
        end = time.process_time()
    print("DFP——精确线搜索:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

## DFP——强Wolf准则

In [20]:
def dfp_wol(fun,gfun,hess,x0):
    #功能：用DFP族算法求解无约束问题：min fun(x)
    #输入：x0是初始点，fun,gfun分别是目标函数和梯度
    #输出：x,val分别是近似最优点和最优解,k是迭代次数
    start = time.process_time()
    maxk = 1e3
    rho = 0.55
    sigma = 0.8
    epsilon = 1e-5
    k = 0
    n = np.shape(x0)[0]
    #海森矩阵可以初始化为单位矩阵
    Hk = np.eye(n) #np.linalg.inv(hess(x0))
 
    while k < maxk :
        gk = gfun(x0)
        if np.linalg.norm(gk) < epsilon:
            break
        dk = -1.0*np.dot(Hk,gk)
 
        m=0;
        mk=0
        while m < 20: 
            #if fun(x0+rho**m*dk) < fun(x0)+sigma*rho**m*np.dot(gk,dk):
            if step_wolfe(fun, gfun, x0, rho**m, dk, 0.3,sigma):
                mk = m
                break
            m += 1
        #print mk
        #DFP校正
        x = x0 + rho**mk*dk
        sk = x - x0
        yk = gfun(x) - gk   
 
        if np.dot(sk,yk) > 0:
            Hy = np.dot(Hk,yk)
            #print (Hy)
            sy = np.dot(sk,yk) #向量的点积
            yHy = np.dot(np.dot(yk,Hk),yk) # yHy是标量
            #表达式Hy.reshape((n,1))*Hy 中Hy是向量，生成矩阵
            #Hk = Hk - 1.0*Hy.reshape((n,1))*Hy/yHy + 1.0*sk.reshape((n,1))*sk/sy
            Hk = Hk - np.dot(Hy.reshape((len(Hy),1)),Hy.reshape((1,len(Hy))))/yHy + np.dot(sk.reshape((len(sk),1)),sk.reshape((1,len(sk))))/sy
 
 
        k += 1
        x0 = x
        end = time.process_time()
    print("DFP——强Wolf准则:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

## DFP——Armijo准则

In [21]:
def dfp_arm(fun,gfun,hess,x0):
    #功能：用DFP族算法求解无约束问题：min fun(x)
    #输入：x0是初始点，fun,gfun分别是目标函数和梯度
    #输出：x,val分别是近似最优点和最优解,k是迭代次数
    start = time.process_time()
    maxk = 1e3
    rho = 0.55
    sigma = 0.8
    epsilon = 1e-5
    k = 0
    n = np.shape(x0)[0]
    #海森矩阵可以初始化为单位矩阵
    Hk = np.eye(n) #np.linalg.inv(hess(x0))
 
    while k < maxk :
        gk = gfun(x0)
        if np.linalg.norm(gk) < epsilon:
            break
        dk = -1.0*np.dot(Hk,gk)
 
        m=0;
        mk=0
        while m < 20: 
            if armijo(gk,dk,x0,rho**m,sigma):
            #if step_wolfe(fun, gfun, x0, rho**m, dk, 0.3,sigma):
                mk = m
                break
            m += 1
        #print mk
        #DFP校正
        x = x0 + rho**mk*dk
        sk = x - x0
        yk = gfun(x) - gk   
 
        if np.dot(sk,yk) > 0:
            Hy = np.dot(Hk,yk)
            #print (Hy)
            sy = np.dot(sk,yk) #向量的点积
            yHy = np.dot(np.dot(yk,Hk),yk) # yHy是标量
            #表达式Hy.reshape((n,1))*Hy 中Hy是向量，生成矩阵
            #Hk = Hk - 1.0*Hy.reshape((n,1))*Hy/yHy + 1.0*sk.reshape((n,1))*sk/sy
            Hk = Hk - np.dot(Hy.reshape((len(Hy),1)),Hy.reshape((1,len(Hy))))/yHy + np.dot(sk.reshape((len(sk),1)),sk.reshape((1,len(sk))))/sy
 
 
        k += 1
        x0 = x
        end = time.process_time()
    print("DFP——Armijo准则:")
    print("运行时间为：",end-start)
    print("终点：",x0)
    print("最小值：",round(fun(x0),3))
    print("迭代次数：",k)
    print('调用Hessian矩阵{}次，调用梯度{}次，调用函数{}次'
                .format(count_G, count_g, count_f))

# Broyden

In [22]:
def broyden_Armijo(fun,gfun,hess,x0):
    #功能：用Broyden族算法求解无约束问题：min fun(x)
    #输入：x0是初始点，fun,gfun分别是目标函数和梯度
    #输出：x,val分别是近似最优点和最优解,k是迭代次数
    start = time.process_time()
    x0 = np.array(x0)
 
    maxk = 1e5
    rho = 0.55;
    sigma = 0.4;
    epsilon = 1e-5
    phi = 0.5;
    k=0;
    n = np.shape(x0)[0]
 
    Hk = np.linalg.inv(hess(x0))
 
    while k<maxk :
        gk = gfun(x0)
        if np.linalg.norm(gk) < epsilon:
            break
        dk = -1*np.dot(Hk,gk)
 
        m=0;mk=0
        while m < 20: 
            # 用Armijo搜索求步长
            if armijo(gk,dk,x0,rho**m,sigma):
                mk = m
                break
            m += 1
        #Broyden族校正
        x = x0 + rho**mk*dk
        sk = x - x0
        yk = gfun(x) - gk
 
        Hy = np.dot(Hk,yk)
        sy = np.dot(sk,yk)
        yHy = np.dot(np.dot(yk,Hk),yk)
 
        if(sy < 0.2 *yHy):
            theta = 0.8*yHy/(yHy-sy)
            sk = theta*sk + (1-theta)*Hy
            sy = 0.2*yHy
 
        vk = np.sqrt(yHy)*(sk/sy-Hy/yHy)
        Hk = Hk - Hy.reshape((n,1))*Hy/yHy +sk.reshape((n,1))*sk/sy + phi*vk.reshape((n,1))*vk
        k += 1
        x0 = x
        end = time.process_time()
    print("运行时间为：",end-start)
    return x0,fun(x0),k #分别是最优点坐标，最优值，迭代次数

# LBFGS

In [23]:
def twoloop(s, y, rho,gk):
    start = time.process_time()
 
    n = len(s) #向量序列的长度
 
    if np.shape(s)[0] >= 1:
        #h0是标量，而非矩阵
        h0 = 1.0*np.dot(s[-1],y[-1])/np.dot(y[-1],y[-1])
    else:
        h0 = 1
 
    a = empty((n,))
 
    q = gk.copy() 
    for i in range(n - 1, -1, -1): 
        a[i] = rho[i] * dot(s[i], q)
        q -= a[i] * y[i]
    z = h0*q
 
    for i in range(n):
        b = rho[i] * dot(y[i], z)
        z += s[i] * (a[i] - b)
 
    return z   
 
def lbfgs_Armijo(fun,gfun,x0,m=5):
    # fun和gfun分别是目标函数及其一阶导数,x0是初值,m为储存的序列的大小
    maxk = 2000
    rou = 0.55
    sigma = 0.4
    epsilon = 1e-5
    k = 0
    n = np.shape(x0)[0] #自变量的维度
 
    s, y, rho = [], [], []
 
    while k < maxk :
        gk = gfun(x0)
        if np.linalg.norm(gk) < epsilon:
            break
 
        dk = -1.0*twoloop(s, y, rho,gk)
 
        m0=0;
        mk=0
        while m0 < 20: 
            # 用Armijo搜索求步长
            if armijo(gk,dk,x0,rho**m,sigma):
                mk = m0
                break
            m0 += 1
 
 
        x = x0 + rou**mk*dk
        sk = x - x0
        yk = gfun(x) - gk   
 
        if np.dot(sk,yk) > 0: #增加新的向量
            rho.append(1.0/np.dot(sk,yk))
            s.append(sk)
            y.append(yk)
        if np.shape(rho)[0] > m: #弃掉最旧向量
            rho.pop(0)
            s.pop(0)
            y.pop(0)
 
        k += 1
        x0 = x
        end = time.process_time()
    print("运行时间为：",end-start)
    return x0,fun(x0),k#分别是最优点坐标，最优值，迭代次数

# 具体函数优化问题

##  Waston函数

In [24]:
count_f = 0 # 计数器，用来统计函数调用的次数
count_g = 0 # 计数器，用来统计梯度调用的次数
count_G = 0 # 计数器，用来统计Hessian矩阵调用的次数
def fun(x):
    global count_f
    count_f += 1 # 每调用一次就将计数器的值增1
    return flambdify(x)

def gfun(y):
    global count_g
    count_g += 1
    return np.array([gf(y) for gf in glambdify])

def hess(y):
    global count_G
    count_G += 1
    return np.array([[gf(y) for gf in Gs] for Gs in Glambdify])
import sympy as sy

m = 31
n = 12  
i, j = sy.symbols('i j')
x = sy.IndexedBase('x')

r1 = sy.Sum((j-1)*x[j-1]*(i/29)**(j-2), (j, 2, n))
r2 = sy.Sum(x[j-1]*(i/29)**(j-1), (j, 1, n))
r = r1 - r2**2 - 1
fexpr = sy.Sum(r**2, (i, 1, m-2)) + x[0]**2 + (x[1] - x[0]**2-1)**2  # 函数表达式
gexpr = [sy.diff(fexpr, x[i]) for i in range(n)] # 梯度函数表达式表
Gexpr = [[sy.diff(g, x[i]) for i in range(n)] for g in gexpr]  # Hessian矩阵表达式的表
### 生成Waston函数及其梯度、Hessian的python数值计算函数，使用numpy库
flambdify = sy.lambdify(x, fexpr, "numpy")  # 函数数值计算
glambdify = [sy.lambdify(x, gf, "numpy") for gf in gexpr]  # 梯度数值计算
Glambdify = [[sy.lambdify(x, gf, "numpy") for gf in Gs] for Gs in Gexpr]  # Hessian矩阵数值计算

In [25]:
dampnm_acc(fun,gfun,hess,np.zeros(n))
dampnm_wol(fun,gfun,hess,np.zeros(n))
dampnm_arm(fun,gfun,hess,np.zeros(n))

阻尼Newton方法——精确线搜索:
运行时间为： 4.596664
终点： [-6.63806085e-09  1.00000164e+00 -5.63932217e-04  3.47820541e-01
 -1.56731505e-01  1.05281518e+00 -3.24727117e+00  7.28843494e+00
 -1.02718483e+01  9.07411368e+00 -4.54137548e+00  1.01201189e+00]
最小值： 0.0
迭代次数： 13
调用Hessian矩阵14次，调用梯度274次，调用函数1次
阻尼Newton方法——强Wolf准则:
运行时间为： 3.2212069999999997
终点： [-6.70374600e-09  1.00000164e+00 -5.63931827e-04  3.47820530e-01
 -1.56731410e-01  1.05281470e+00 -3.24726970e+00  7.28843211e+00
 -1.02718448e+01  9.07411111e+00 -4.54137440e+00  1.01201170e+00]
最小值： 0.0
迭代次数： 13
调用Hessian矩阵28次，调用梯度387次，调用函数68次
阻尼Newton方法——Armijo准则:
运行时间为： 2.362143999999999
终点： [-6.88394584e-09  1.00000164e+00 -5.63932367e-04  3.47820537e-01
 -1.56731467e-01  1.05281497e+00 -3.24727050e+00  7.28843360e+00
 -1.02718466e+01  9.07411241e+00 -4.54137494e+00  1.01201179e+00]
最小值： 0.0
迭代次数： 13
调用Hessian矩阵42次，调用梯度401次，调用函数97次


从阻尼牛顿方法输出结果看，三种线搜索方法迭代次数一致，但是尽管非精确线搜索方法调用海森矩阵、梯度、函数的计算次数更多，但非精确线搜索方法确定步长的环节更加简洁，因而整体的运行时间反而更短

In [26]:
revisenm_acc(fun,gfun,hess,np.zeros(n))
revisenm_wol(fun,gfun,hess,np.zeros(n))
revisenm_arm(fun,gfun,hess,np.zeros(n))

修正Newton方法——精确线搜索:
运行时间为： 4.8159980000000004
终点： [-3.17039378e-06  9.99959043e-01  1.73611378e-03  3.16137946e-01
  6.90198937e-02  1.42212798e-02  6.09773191e-02  7.96576744e-02
  3.34881427e-02 -3.79372867e-02 -5.47907447e-02  7.49385662e-02]
最小值： 0.0
迭代次数： 14
调用Hessian矩阵56次，调用梯度696次，调用函数98次
修正Newton方法——强Wolf准则:
运行时间为： 5.279613000000001
终点： [-3.17039378e-06  9.99959043e-01  1.73611378e-03  3.16137946e-01
  6.90198937e-02  1.42212798e-02  6.09773191e-02  7.96576744e-02
  3.34881427e-02 -3.79372867e-02 -5.47907447e-02  7.49385662e-02]
最小值： 0.0
迭代次数： 14
调用Hessian矩阵70次，调用梯度1038次，调用函数317次
修正Newton方法——Armijo准则:
运行时间为： 2.494347000000001
终点： [-3.17039378e-06  9.99959043e-01  1.73611378e-03  3.16137946e-01
  6.90198937e-02  1.42212798e-02  6.09773191e-02  7.96576744e-02
  3.34881427e-02 -3.79372867e-02 -5.47907447e-02  7.49385662e-02]
最小值： 0.0
迭代次数： 14
调用Hessian矩阵84次，调用梯度1053次，调用函数346次


从修正牛顿方法输出结果看，三种线搜索方法迭代次数一致，但是强Wolfe准则调用海森矩阵、梯度、函数的计算次数更多，尽管非精确线搜索更加省时间，也仍然整体的运行时间也更长；但是Armijo准则的判断条件相比于Wolfe准则更少，运行时间最短。

In [27]:
sr1_acc(fun,gfun,hess,np.zeros(n))
sr1_wol(fun,gfun,hess,np.zeros(n))
sr1_arm(fun,gfun,hess,np.zeros(n))

SR1——精确线搜索:
运行时间为： 63.9418
终点： [-3.58115433e-08  1.00000165e+00 -5.63335302e-04  3.47812261e-01
 -1.56674015e-01  1.05257968e+00 -3.24666368e+00  7.28741686e+00
 -1.02707379e+01  9.07335097e+00 -4.54107514e+00  1.01196010e+00]
最小值： 0.0
迭代次数： 352
调用Hessian矩阵84次，调用梯度8798次，调用函数347次
SR1——强Wolf准则:
运行时间为： 7.419729000000004
终点： [-3.00096038e-06  1.00002357e+00  3.92521793e-04  3.23231871e-01
  5.51013521e-02  2.03244904e-02  7.08144466e-02  7.63619724e-02
  2.33551731e-02 -3.87248416e-02 -4.33549333e-02  6.98779741e-02]
最小值： 0.0
迭代次数： 34
调用Hessian矩阵84次，调用梯度9671次，调用函数884次
SR1——Armijo准则:
运行时间为： 0.8830800000000067
终点： [-3.09072314e-06  1.00002382e+00  3.85974703e-04  3.23274576e-01
  5.49922281e-02  2.04166989e-02  7.08575626e-02  7.63035374e-02
  2.32959665e-02 -3.87018795e-02 -4.32808997e-02  6.98358141e-02]
最小值： 0.0
迭代次数： 44
调用Hessian矩阵84次，调用梯度9760次，调用函数1313次


从SR1方法输出结果看，精确线搜索方法的迭代次数明显最大，两种非精确线搜索方法中强Wolfe准则迭代次数更少，使用Armijo准则运行时间最短，使用精确线搜索方法运行时间最长。

In [28]:
#bfgs_acc(fun,gfun,hess,np.zeros(n))
bfgs_wol(fun,gfun,hess,np.zeros(n))
bfgs_arm(fun,gfun,hess,np.zeros(n))

BFGS——强Wolf准则:
运行时间为： 11.569340999999994
终点： [-2.93931325e-06  1.00002335e+00  3.95976754e-04  3.23211039e-01
  5.51539163e-02  2.02816334e-02  7.07889135e-02  7.63911027e-02
  2.33902162e-02 -3.87355501e-02 -4.34000627e-02  6.99031577e-02]
最小值： 0.0
迭代次数： 44
调用Hessian矩阵84次，调用梯度11118次，调用函数2160次
BFGS——Armijo准则:
运行时间为： 0.7663869999999946
终点： [-3.00088825e-06  1.00002354e+00  3.93307016e-04  3.23228086e-01
  5.51072735e-02  2.03243990e-02  7.08090185e-02  7.63612352e-02
  2.33593323e-02 -3.87228761e-02 -4.33585283e-02  6.98788221e-02]
最小值： 0.0
迭代次数： 42
调用Hessian矩阵84次，调用梯度11203次，调用函数2325次


从BFGS方法输出结果看，精确线搜索无法收敛，此处略去结果。其他两种非精确线搜索方法中Armijo准则迭代次数更少，尽管Armijo准则调用梯度、函数的计算次数更多但其判断步长所需条件更少，仍然整体的运行时间也更短

In [29]:
dfp_acc(fun,gfun,hess,np.zeros(n))
dfp_wol(fun,gfun,hess,np.zeros(n))
dfp_arm(fun,gfun,hess,np.zeros(n))

DFP——精确线搜索:
运行时间为： 181.894756
终点： [nan nan nan nan nan nan nan nan nan nan nan nan]
最小值： nan
迭代次数： 1000
调用Hessian矩阵84次，调用梯度33203次，调用函数2326次
DFP——强Wolf准则:
运行时间为： 43.03223200000002
终点： [ 1.33113065e-04  9.99910792e-01  4.37708561e-03  3.05481764e-01
  7.62071194e-02  2.82509314e-02  5.85953379e-02  6.17720028e-02
  2.16792853e-02 -2.61319212e-02 -3.02298070e-02  5.77870200e-02]
最小值： 0.0
迭代次数： 1000
调用Hessian矩阵84次，调用梯度38305次，调用函数4395次
DFP——Armijo准则:
运行时间为： 18.667880000000025
终点： [-5.55217638e-04  1.00021149e+00 -3.89553602e-03  3.36530989e-01
  4.38871926e-02  1.34565658e-02  7.38114252e-02  8.30439499e-02
  2.67734908e-02 -4.11689003e-02 -4.82734744e-02  7.17781364e-02]
最小值： 0.0
迭代次数： 1000
调用Hessian矩阵84次，调用梯度40305次，调用函数10460次


从DFP方法输出结果看，精确线搜索方法1000步内未收敛，强Wolf准则下调用海森矩阵、梯度、函数的计算次数更少，但Armijo准则下整体的运行时间也更短

##  Discrete Boundary Value

In [30]:
count_f = 0 # 计数器，用来统计函数调用的次数
count_g = 0 # 计数器，用来统计梯度调用的次数
count_G = 0 # 计数器，用来统计Hessian矩阵调用的次数
def fun(x):
    global count_f
    count_f += 1 # 每调用一次就将计数器的值增1
    return flambdify(x)

def gfun(y):
    global count_g
    count_g += 1
    return np.array([gf(y) for gf in glambdify])

def hess(y):
    global count_G
    count_G += 1
    return np.array([[gf(y) for gf in Gs] for Gs in Glambdify])
import sympy as sy

m = 31
n = 31  
i, j = sy.symbols('i j')
x = sy.IndexedBase('x')
r = 2*x[i-1]-x[i-2]-x[i]+((1/(n+1))**2)*((x[i-1]+i*(1/(n+1))+1)**3)/2
fexpr = (2*x[0]-x[1]+((1/(n+1))**2)*((x[0]+(1/(n+1))+1)**3)/2)**2+sy.Sum(r**2, (i, 2, m-1))+(2*x[n-1]-x[n-2]+((1/(n+1))**2)*((x[n-1]+n*(1/(n+1))+1)**3)/2)**2 # 函数表达式
gexpr = [sy.diff(fexpr, x[i]) for i in range(n+2)] # 梯度函数表达式表
Gexpr = [[sy.diff(g, x[i]) for i in range(n+2)] for g in gexpr]  # Hessian矩阵表达式的表

flambdify = sy.lambdify(x, fexpr, "numpy")  # 函数数值计算
glambdify = [sy.lambdify(x, gf, "numpy") for gf in gexpr]  # 梯度数值计算
Glambdify = [[sy.lambdify(x, gf, "numpy") for gf in Gs] for Gs in Gexpr]  # Hessian矩阵数值计算

In [31]:
h=1/(n+1)
ti=[i*h for i in list(range(0,n+1))]
x0=[t*(t-1) for t in ti]
x0.append(0)
ti.append(0)

In [32]:
#dampnm_acc(fun,gfun,hess,x0)
#dampnm_wol(fun,gfun,hess,x0)
#dampnm_arm(fun,gfun,hess,x0)

从阻尼牛顿方法输出结果看，此时算法无法收敛。

In [33]:
revisenm_acc(fun,gfun,hess,x0)
revisenm_wol(fun,gfun,hess,x0)
revisenm_arm(fun,gfun,hess,x0)

修正Newton方法——精确线搜索:
运行时间为： 5.050370999999984
终点： [-0.01559256 -0.03067072 -0.0452069  -0.05917191 -0.07253489 -0.08526312
 -0.09732191 -0.10867441 -0.11928142 -0.12910113 -0.13808892 -0.14619703
 -0.15337426 -0.15956558 -0.16471173 -0.16874875 -0.17160746 -0.17321283
 -0.17348331 -0.17233008 -0.16965615 -0.16535534 -0.15931116 -0.15139545
 -0.14146687 -0.12936915 -0.11492901 -0.09795381 -0.07822871 -0.05551343
 -0.02953837 -0.03027344  0.        ]
最小值： 0.0
迭代次数： 4
调用Hessian矩阵4次，调用梯度85次，调用函数1次
修正Newton方法——强Wolf准则:
运行时间为： 4.476517000000001
终点： [-0.01559256 -0.03067072 -0.0452069  -0.05917191 -0.07253489 -0.08526312
 -0.09732191 -0.10867441 -0.11928142 -0.12910113 -0.13808892 -0.14619703
 -0.15337426 -0.15956558 -0.16471173 -0.16874875 -0.17160746 -0.17321283
 -0.17348331 -0.17233008 -0.16965615 -0.16535534 -0.15931116 -0.15139545
 -0.14146687 -0.12936915 -0.11492901 -0.09795381 -0.07822871 -0.05551343
 -0.02953837 -0.03027344  0.        ]
最小值： 0.0
迭代次数： 4
调用Hessian矩阵8次，调用梯度102次，调用函数10次
修正

从修正牛顿方法输出结果看，三种线搜索方法迭代次数一致，但是尽管Armijo准则下调用海森矩阵、梯度、函数的计算次数更多，也仍然整体的运行时间最短；而精确线搜索下运行时间最长。

In [34]:
sr1_acc(fun,gfun,hess,x0)
sr1_wol(fun,gfun,hess,x0)
sr1_arm(fun,gfun,hess,x0)

SR1——精确线搜索:
运行时间为： 17.71779600000002
终点： [-0.01536886 -0.03022587 -0.04454593 -0.05830221 -0.07146605 -0.08400684
 -0.09589177 -0.10708575 -0.11755107 -0.12724726 -0.13613073 -0.14415455
 -0.15126811 -0.15741677 -0.16254143 -0.16657811 -0.16945734 -0.17110363
 -0.17143481 -0.17036125 -0.16778499 -0.16359875 -0.15768478 -0.14991357
 -0.14014231 -0.12821319 -0.11395131 -0.09716235 -0.0776298  -0.05511169
 -0.02933678 -0.03027344  0.        ]
最小值： 0.0
迭代次数： 37
调用Hessian矩阵12次，调用梯度922次，调用函数20次
SR1——强Wolf准则:
运行时间为： 17.729196
终点： [-0.01531309 -0.03011674 -0.04438818 -0.05810274 -0.07123359 -0.08375148
 -0.0956245  -0.10681783 -0.11729352 -0.12701031 -0.1359234  -0.14398429
 -0.15114053 -0.15733545 -0.16250791 -0.16659186 -0.16951598 -0.1712031
 -0.17156965 -0.17052492 -0.16797018 -0.16379776 -0.15788983 -0.15011708
 -0.14033716 -0.12839289 -0.11411014 -0.09729544 -0.07773317 -0.05518233
 -0.02937263 -0.03027344  0.        ]
最小值： 0.0
迭代次数： 36
调用Hessian矩阵12次，调用梯度1763次，调用函数533次
SR1——Armijo准则:
运行

从SR1方法输出结果看，三种线搜索方法迭代次数相近，与修正牛顿方法结果相似，Armijo准则下调用梯度、函数的计算次数更多，也仍然整体的运行时间最短；而精确线搜索下运行时间与强Wolf准则下相近。 

In [35]:
#bfgs_acc(fun,gfun,hess,x0)
bfgs_wol(fun,gfun,hess,x0)
bfgs_arm(fun,gfun,hess,x0)

BFGS——强Wolf准则:
运行时间为： 38.478836
终点： [-0.01542965 -0.03034517 -0.04471921 -0.05852241 -0.07172362 -0.08429065
 -0.09619019 -0.10738743 -0.11784584 -0.12752684 -0.13638919 -0.1443881
 -0.15147453 -0.15759484 -0.16269052 -0.16669814 -0.16954924 -0.17116971
 -0.17147905 -0.17038913 -0.16780269 -0.16361219 -0.15769855 -0.14993028
 -0.14016258 -0.12823604 -0.11397473 -0.09718384 -0.07764715 -0.05512358
 -0.02934285 -0.03027344  0.        ]
最小值： 0.0
迭代次数： 57
调用Hessian矩阵12次，调用梯度3574次，调用函数2099次
BFGS——Armijo准则:
运行时间为： 2.645175999999992
终点： [-0.01538725 -0.03026195 -0.04459801 -0.05836814 -0.07154309 -0.08409165
 -0.09598088 -0.10717564 -0.11763838 -0.12732911 -0.13620466 -0.14421875
 -0.15132132 -0.15745831 -0.16257074 -0.16659512 -0.16946242 -0.17109776
 -0.17141928 -0.17033791 -0.16775587 -0.16356612 -0.15765079 -0.14988017
 -0.14011088 -0.12818469 -0.11392644 -0.09714162 -0.07761381 -0.05510069
 -0.02933123 -0.03027344  0.        ]
最小值： 0.0
迭代次数： 58
调用Hessian矩阵12次，调用梯度3691次，调用函数2396次


从BFGS方法输出结果看，精确线搜索方法未收敛，两种非精确线搜索方法迭代次数相近，Armijo准则下调用梯度、函数的计算次数更多，也仍然整体的运行时间明显短于强Wolf准则下。

In [36]:
dfp_acc(fun,gfun,hess,x0)
dfp_wol(fun,gfun,hess,x0)
dfp_arm(fun,gfun,hess,x0)

DFP——精确线搜索:
运行时间为： 86.76857599999994
终点： [-0.02493616 -0.04610473 -0.06173676 -0.06962496 -0.07309095 -0.0807263
 -0.09483683 -0.1120857  -0.13346903 -0.16050815 -0.19150126 -0.22017726
 -0.24361783 -0.25650093 -0.26056508 -0.26415946 -0.27269394 -0.28629272
 -0.30216574 -0.32209456 -0.3461552  -0.36478577 -0.37331755 -0.37134848
 -0.3571527  -0.3270232  -0.28433818 -0.23488618 -0.18128231 -0.12609672
 -0.0666391  -0.03027344  0.        ]
最小值： 0.001
迭代次数： 1000
调用Hessian矩阵12次，调用梯度25691次，调用函数2397次
DFP——强Wolf准则:
运行时间为： 19.913798999999926
终点： [-0.01604544 -0.03123856 -0.04613978 -0.0611276  -0.07576902 -0.09001191
 -0.10404993 -0.1179802  -0.13117001 -0.1435818  -0.15493756 -0.16491385
 -0.17356135 -0.18052427 -0.18536367 -0.18849139 -0.19010644 -0.19015278
 -0.18821765 -0.1847968  -0.1801721  -0.17403112 -0.16638725 -0.15714271
 -0.14599478 -0.1331528  -0.11814304 -0.1004729  -0.07994263 -0.05679759
 -0.03042603 -0.03027344  0.        ]
最小值： 0.0
迭代次数： 1000
调用Hessian矩阵12次，调用梯度30808次，调用函数44

从DFP方法输出结果看，精确线搜索方法还未收敛，两种非精确线搜索方法中，Armijo准则下调用梯度、函数的计算次数更多，也仍然整体的运行时间明显短于强Wolf准则下，且仍然是精确线搜索方法下运行时间明显最长。

# 总结

**<font size=3>（1）综合上述比较结果，可以看出尽管非精确线搜索方法尤其是Armijo准则在部分方法中可能会调用更多次数的海森矩阵或者梯度或者函数，但是简化了搜索步长的环节反而有明显最短的运行时间。尤其是精确线搜索的运行时间大多明显较长，甚至会出现1000步内不收敛的情况。在不同的优化算法中，三种步长的搜索准则表现差异较大，因此在具体选择步长的确定方法的时候，还是要针对不同的具体优化方法进行选择。</font>**
    
**<font size=3>（2）并且有的情况下，阻尼牛顿方法会因为矩阵奇异而无法收敛，说明目前来看，最常使用的还是拟牛顿方法是具有一定道理的。</font>**

**<font size=3>（3）本次大作业中也编写了其他题目为要求的优化算法函数，后续也可以尝试对其他方法一起进行比较。</font>**